# Computação quântica
## Conceitos básicos 

- bits quânticos (qbit)
- operadores sobre um qbit
- Medição
- Simulação utilizando qiskit
- Execução em um processador quântico


## Introdução
A matéria em escala subatômica apresenta um comportamento diferente do descrito na física clássica. A computação quântica utiliza propriedades inerentes a matéria em escala subatômica para realizar processamento da informação. Os únicos pré-requisitos deste curso são álgebra linear, algoritmos e estruturas de dados. 

A computação clássica processa bits que podem estar no "estado" 0 ou 1. Por outro lado a computação quântica processa bits quânticos ou **qbits** podem estar em um número contínuo de possibilidades. Utilizando a computação quântica é possível fatorar um número em tempo polinomial e realizar uma busca em um conjunto desordenado com custo $O(\sqrt{n})$.

O principal objetivo deste curso é apresentar os conceitos da computação quântica, os principais algoritmos e métodos de programação para processadores quânticos de pequena escala.

Diversas empresas, universidades e países estão realizando investimentos para a construção de um computador quântico. Apesar dos resultados teóricos, até a data desta aula ainda não existe uma aplicação prática de computadores quânticos que não possa ser realizada por computadores clássicos.

## Bits quânticos
Na literatura de computação quântica a notação de dirac $|\cdot \rangle$ é utilizada para representar vetores. Por exemplo,

$$|0\rangle = \begin{bmatrix}1 \\ 0\end{bmatrix} \mbox{ e } |1\rangle = \begin{bmatrix}0 \\ 1\end{bmatrix}.$$

Enquanto os bits podem estar em apenas dois possíveis estados, os bits quânticos podem estar em uma superposição de estados (em uma combinação linear de $|0\rangle$ e $|1\rangle$) $$|\psi\rangle = \alpha|0\rangle + \beta|1\rangle,$$.

onde $\alpha$ e $\beta$ são denominados amplitudes de probabilidade e $|\alpha|^2+|\beta|^2 = 1$


## Medição projetiva

Os valores das amplitudes de probabilidade de um qbit não podem ser recuperados. A única informação que obtemos ao observar um qbit $|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$ é 0 com probabilidade $|\alpha|^2$ e 1 com probabilidade $|\beta|^2$. Após a medição o estado do qbit colapsa e sucessivas medições retornarão o mesmo resultado.

Por exemplo, considere o qbit $|\psi\rangle = \frac{1}{\sqrt{3}}|0\rangle + \sqrt{\frac{2}{3}}|1\rangle$. Ao realizar uma medição será obtido 0 com probabilidade $\frac{1}{3}$ e neste caso o qbit irá colapsar para o estado $|\psi\rangle =|0\rangle$ ou 1 com probabilidade $\frac{2}{3}$ e neste caso o qbit irá colapsar para o estado $|\psi\rangle =|1\rangle$.

## Operadores quânticos sobre um qbit
Na base computacional (ou base canônica) $\{|0\rangle, |1\rangle\}$, um operador quântico sobre um qbit pode ser representado por uma matriz unitária $2 \times 2$. Dizemos que uma matriz $U$ é unitária se ao ser multiplicada por sua transposta conjugada $U^\dagger$ obtiver a matriz identidade como resultado $UU^\dagger = I$. Toda matriz unitária $2 \times 2$ representa um operador quântico válido.

Alguns exemplos de operadores quânticos $I$, $H$, $X$, $Z$, 

$$I = \begin{bmatrix}
1 & 0 \\
0 & 1
\end{bmatrix}\mbox{, } 
%
X = \begin{bmatrix}
0 & 1 \\
1 & 0
\end{bmatrix}\mbox{, }
%
H = \frac{1}{\sqrt{2}}\begin{bmatrix}
1 & 1 \\
1 & -1
\end{bmatrix}\mbox{ e }
Z = \frac{1}{\sqrt{2}}\begin{bmatrix}
1 & 0 \\
0 & -1
\end{bmatrix}$$

Toda matriz $2 \times 2$ unitária representa um operador quântico válido. No entanto, os processadores quânticos disponíveis possuem um conjunto finito de funções para representar operadores quânticos. Por exemplo, no computador ibmqx4 o conjunto de operadores sobre um qbit são

$$u3(\theta, \phi, \lambda) = 
\begin{bmatrix} 
    cos(\theta/2) & -e^{i\lambda}sen(\theta/2) \\
    e^{i\phi}sen(\theta/2) & e^{i(\lambda+\phi)}cos(\theta/2)
\end{bmatrix}\mbox{, } 
u2(\phi, \lambda)=
\begin{bmatrix}
1 & -e^{i\lambda}\\
e^{i\phi} & e^{i\phi + i\lambda}
\end{bmatrix}, 
u1(\lambda)=
\begin{bmatrix}
1 & 0\\
0 & e^{i\lambda}
\end{bmatrix}
$$

## Simulação utilizando o numpy

Podemos pensar em um bit quântico como uma classe com um atributo privado vetor. O vetor é inicializado em $|0\rangle$ e só pode ser modificado pela ação de operadores quânticos e medições.

In [1]:
import numpy as np
import random
class qbit:
    def __init__(self):
        self._vector = np.matrix([[1],[0]]) # _ indica aproximadamente variável protegida
    def measure(self):
        a = random.random()
        if a < self._vector[0]**2:
            self._vector = np.matrix([[1],[0]])
            return 0
        else:
            self._vector = np.matrix([[0],[1]])
            return 1
    def __str__(self):
        return str(self._vector[0]) + '|0>' + ' + ' + str(self._vector[1]) + '|1>'
    
class qop:
    def __init__(self, matrix):
        self.matrix = np.matrix(matrix)
    def __mul__(self, qbit):
        qbit._vector = self.matrix * qbit._vector
    
    def __str__(self):
        return str(self.matrix)

In [15]:
# inicializa um qbit
psi = qbit() # todos qbits s

# inicializa um operador quântico
a = (1/np.sqrt(2)) * np.matrix([[1, 1],[1, -1]])
H = qop(a)

#Aplica o operador ao bit quântico
H * psi

In [16]:
print(psi) # esta operação não é possível em qbit real

[[0.70710678]]|0> + [[0.70710678]]|1>


In [17]:
# Medição do qbit
psi.measure()

0

In [18]:
# Estado após a medição
print(psi) # esta operação não é possível em qbit real

[[1]]|0> + [[0]]|1>


## Qiskit
O [qiskit](https://qiskit.org) fornece ferramentas para a programação de computadores quânticos na linguagem de programação python. Para inicializar um qbit utilizamos a função ```QuantumRegister```, para inicializar um bit clássico utilizamos a função ```ClassicalRegister``` e é necessário construir um circuito quântico com a classe `python QuantumCircuit` para realizar operações sobre os qbits. Estes são os objetos básicos para programar um sistema quântico.

```python 
QuantumRegister(size, name=None) 
# implementa um registrador quântico como um array de qbits

ClassicalRegister(size, name=None) 
# implementa um registrador clássico como um array de qbits, size inteiro, name string

QuantumCircuit(*regs, name=None) 
# Cria um circuito quântico. *regs lista de ClassicalRegisters e Quantum registers e name string
```

No código abaixo um registrador quântico `q`, um registrador clássico `c` e um circuito quântico `circuit` contendo `q` e `c` são criados. Em seguida o operador Hadammard `circuit.h(q)` é aplicado no registrador `q` e uma medição do registrador `q` é realizada e seu resultado é armazenado no registrador `c`.

In [6]:
# importar módulos
import numpy as np
import qiskit

q = qiskit.QuantumRegister(1, 'q')
c = qiskit.ClassicalRegister(1, 'c')
circuit = qiskit.QuantumCircuit(q, c)
circuit.h(q)
circuit.measure(q, c)



Podemos visualizar o circuito `circuit`utilizando a função `draw()` da classe `QuantumCircuit`. Um circuito é lido da esquerda para direita. No circuito abaixo o operador $H$ é aplicado para em seguida realizarmos uma medição.

In [7]:
circuit.draw()


Para executar um circuito quântico devemos utilizar o módulo `BasicAer`. A função `BasicAer.get_backend` permite selecionar onde o circuito será executado e a função `qiskit.execute` para executar o circuito.


In [8]:
from qiskit import BasicAer
backend = BasicAer.get_backend('qasm_simulator')
job = qiskit.execute(circuit, backend, shots=1024)

In [9]:
result = job.result()
counts = result.get_counts(circuit)
print(counts)

{'0': 489, '1': 535}


## Executando o circuito em um processador quântico

A IBM disponibiliza protótipos de processadores quânticos com 5 qbits (ibmq_5_yorktown - ibmqx2 e ibmq_5_tenerife - ibmqx4) e com 14 qbits (ibmq_16_melbourne). Para acessar estes computadores é necessário criar uma conta em https://quantum-computing.ibm.com/login.

Após criar uma conta é necessário acessar perfil $\rightarrow$ my account $\rightarrow$ copy token. Esse token é pessoal e não deve ser compartilhado. Ao executar a célula abaixo informe seu token quando solicitado para registrar este notebook.

In [10]:
from qiskit import IBMQ 
import getpass

IBMQ.backends()

[]

In [11]:
MY_API_TOKEN = getpass.getpass("Informe seu API_TOKEN: ")

Informe seu API_TOKEN: ········


In [12]:
IBMQ.enable_account(MY_API_TOKEN)
IBMQ.backends()

[<IBMQBackend('ibmqx4') from IBMQ()>,
 <IBMQBackend('ibmqx2') from IBMQ()>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ()>,
 <IBMQSimulator('ibmq_qasm_simulator') from IBMQ()>]

In [13]:
IBMQ.backends(operational=True)

[<IBMQBackend('ibmqx4') from IBMQ()>,
 <IBMQSimulator('ibmq_qasm_simulator') from IBMQ()>]

In [14]:
qbackend = IBMQ.get_backend('ibmqx4')
qjob = qiskit.execute(circuit, qbackend, shots=1024)
qresult = qjob.result()
qcounts = qresult.get_counts(circuit)
print(qcounts)

{'0': 583, '1': 441}


## Exercícios

1. Quantos bits podem ser armazenados em um qbit?
1. Verifique que o operador $H$ é unitário.
1. Verifique que $H^2 = I$
1. Quais são os autovalores e autovetores de $H$?
1. Determine o estado do qbit $|\psi\rangle = ZHX|0\rangle$. Verifique a probabilidade de obtermos 0 ou 1 ao realizar a medição do qbit e simule as operações utilizando o qiskit para verificar sua estimativa.

1. Implemente um circuito quântico que crie a partir do $|0\rangle$ o estado $\frac{1}{\sqrt{7}}|0\rangle - \frac{\sqrt{6}}{\sqrt{7}}|1\rangle$

1. Utilizando o numpy, mostre que $HXH = Z$, $HYH = -Y$ e $HZH=X$